# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [10]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2009-10-01	Paper Title Number 1	Journal 1	This paper is about the number 1. The number 2 is left for future work.	Your Name, You. (2009). "Paper Title Number 1." <i>Journal 1</i>. 1(1).	paper-title-number-1	http://academicpages.github.io/files/paper1.pdf	http://academicpages.github.io/files/slides1.pdf
2010-10-01	Paper Title Number 2	Journal 1	This paper is about the number 2. The number 3 is left for future work.	Your Name, You. (2010). "Paper Title Number 2." <i>Journal 1</i>. 1(2).	paper-title-number-2	http://academicpages.github.io/files/paper2.pdf	http://academicpages.github.io/files/slides2.pdf
2015-10-01	Paper Title Number 3	Journal 1	This paper is about the number 3. The number 4 is left for future work.	Your Name, You. (2015). "Paper Title Number 3." <i>Journal 1</i>. 1(3).	paper-title-number-3	http://academicpages.github.io/files/paper3.pdf	http://academicpages.github.io/files/slides3.pdf

## Import pandas

We are using the very handy pandas library for dataframes.

In [1]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [45]:
#publications = pd.read_csv("publications.csv", sep=",", header=0)
publications = pd.read_csv("confpublications.csv", sep=",", header=0)

publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2024-16-07,Design of an autonomous ag platform capable of...,Proceedings of the 16th Proceedings of the 16t...,"Pivot+ is intended to serve as an innovative, ...","Balmos A. D., Jha S., Krogmeier J. V., Buckmas...","Balmos A. D., Jha S., Krogmeier J. V., Buckmas...",http://Snehajha.github.io/files/Design_of_an_A...,NaN
1,2024-16-07,OATSmobile: a Data Hub for Underground Sensor ...,Proceedings of the 16th Proceedings of the 16t...,Wireless Underground Sensor Networks (WUSNs) p...,"Brinton C., Wang C., & Cappelleri D. (2024). D...","Brinton C., Wang C., & Cappelleri D. (2024). D...",http://Snehajha.github.io/files/OATSmobile_a_D...,NaN
2,7/1/18,Analyzing Trends for Agricultural Decision Sup...,Proceedings of the 14th Proceedings of the 16t...,The trends and reactions of the general public...,"ha S., Saraswat D., and Ward M.D. “Trends in B...","ha S., Saraswat D., and Ward M.D. “Trends in B...",http://Snehajha.github.io/files/Analyzing_Tren...,NaN
3,NaN,NaN,NaN,NaN,systems using Twitter data.”(2018 ). 14th Inte...,systems using Twitter data.”(2018 ). 14th Inte...,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [46]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [47]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

TypeError: 'float' object is not subscriptable

These files are in the publications directory, one directory below where we're working from.

In [48]:
!ls ../_publications/

18-ha S., Saraswat D., and Ward M.D. “Trends in Big Data solutions in Agricultural decision support.md
2009-10-01-paper-title-number-1.md
2024-16-07-Balmos A. D., Jha S., Krogmeier J. V., Buckmaster D. R., Love D. J., Grant R. H., Crawford M.,.md
2024-16-07-Brinton C., Wang C., & Cappelleri D. (2024). Design of an autonomous ag platform capable.md
21-sunsimulator.md
23-Jha, S., Zhang, Y., Park, B., Cho, S., Krogmeier, J. V., Bagchi, T., & Haddock, J. E. (2023). Data-Driven Web-Based Patching Management Tool Using Multi-Sensor Pavement Structure Measurements. Transportation Research Record, 2677(12), 83-98..md
24-Jha, S., Krogmeier, J. V., Buckmaster, D. R., & Balmos, A. D. (2020). Python Programming in Digital Agriculture. In Case Studies and Modules for Data Science Instruction (pp. 7-24). American Society of Agricultural and Biological Engineers..md


In [49]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

---
title: "Paper Title Number 1"
collection: publications
category: manuscripts
permalink: /publication/2009-10-01-paper-title-number-1
excerpt: 'This paper is about the number 1. The number 2 is left for future work.'
date: 2009-10-01
venue: 'Journal 1'
slidesurl: 'http://academicpages.github.io/files/slides1.pdf'
paperurl: 'http://academicpages.github.io/files/paper1.pdf'
citation: 'Your Name, You. (2009). &quot;Paper Title Number 1.&quot; <i>Journal 1</i>. 1(1).'
---

The contents above will be part of a list of publications, if the user clicks the link for the publication than the contents of section will be rendered as a full page, allowing you to provide more information about the paper for the reader. When publications are displayed as a single page, the contents of the above "citation" field will automatically be included below this section in a smaller font.